In [ ]:
!pip install fastapi
!pip install colabcode
!pip install detoxify
!pip install transformers
import torch
from transformers import BertTokenizer
from detoxify import toxic_bert
import time

In [ ]:
from fastapi import FastAPI
import pickle
import numpy as np
app = FastAPI()

In [ ]:
from colabcode import ColabCode
server = ColabCode(port = 10000, code = False)

In [ ]:
from detoxify import toxic_bert
model = toxic_bert().cuda()
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
# checking the 
start = time.time()
temp_messages = ["sample message sample message sample message sample message sample message sample message  "]*1000

temp_1 = []
for i in temp_messages:
    temp = tokenizer(i,return_tensors="pt")
    temp_1.append(temp)

text = []
attention = []
for i in temp_1:
    text.append(np.array(i["input_ids"][0]))
    attention.append(np.array(i["attention_mask"][0]))

text = torch.IntTensor(text).cuda()
attention = torch.IntTensor(attention).cuda()
temp_model = None
with torch.no_grad():
    temp_model = model(text,attention)
print(temp_model.logits.shape)
print(temp_model.logits[:,0].shape)
print("Total time taken ", time.time() - start)

torch.Size([1000, 6])
torch.Size([1000])
Total time taken  0.8549494743347168


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  from ipykernel import kernelapp as app


In [ ]:
from collections import defaultdict 
def get_predictions(input_message):
    input_message = input_message.split("-`!^")
    print()
    dict1 = defaultdict(list)
    for i in input_message:
        if i!= '':
          user = i.split('-`!~')
          if len(user)>1:
            dict1[user[0]].append(user[1])


    dict2 = {k: v for k, v in sorted(dict1.items(), key=lambda item: -len(item[1]))}
    
    count_5 = 0 
    temp_1 = []
    for i in dict2.values():
        
        temp = tokenizer(i,return_tensors="pt",padding='max_length', truncation = True, max_length= 130)
        temp_1.append(temp)
        count_5 +=1
        if count_5 == 5:
          break

    text = []
    attention = []
    for i in temp_1:
        text.append(np.array(i["input_ids"][0]))
        attention.append(np.array(i["attention_mask"][0]))

    text = torch.IntTensor(text).cuda()
    attention = torch.IntTensor(attention).cuda()
    
    listofusers =[]
    for i in dict2.keys():
        listofusers.append(i)
    with torch.no_grad():
        model_output=torch.nn.Softmax(dim = 1)(model(text,attention).logits)
    
    output = {}
    for i in range(len(model_output)):
        output[listofusers[i]] = model_output[i].tolist()
        output[listofusers[i]].append(len(dict2[listofusers[i]]))
    
    return output

In [ ]:
get_predictions('''-`!^chevalier_au_lion_-`!~monkaW-`!^fixyouract-`!~DinkDonk-`!^drov_-`!~thought tha twas outside-`!^jasminehollyxo-`!~DinkDonk-`!^gokumeister-`!~PogU rolling-`!^thesilencer99-`!~DinkDonk-`!^elliot_xqcl-`!~pepeMeltdown BEEEEP-`!^trippinovar-`!~?-`!^p0werr4ngerx-`!~DinkDonk-`!^kratos943-`!~DinkDonk-`!^karimtv-`!~this game is sick-`!^meidiemalding-`!~AlienPls3-`!^17sirio-`!~my car bro...-`!^martinezvibesss-`!~xqcOmega-`!^zxxst-`!~Don’t put too much faith in the pump… Top 5 it’s terrible-`!^notcluster_-`!~@crimsonwinson how is that huh-`!^linark__-`!~DinkDonk-`!^sandfir1-`!~physikz too many people get tapped for being risque in chats OMEGALUL-`!^brumakw-`!~BABACIII-`!^inmind02-`!~DinkDonk-`!^tealins-`!~kripcDink-`!^pedrommferreira-`!~MONGAW-`!^gogomatiu-`!~Wokege-`!^guichi6-`!~hit and run 🤯-`!^marvindamartian23-`!~U GOT THIS-`!^doseboy287-`!~DinkDonk-`!^tuica69-`!~DinkDonk-`!^amariel54321-`!~you got this xQCOW-`!^niquexpress-`!~xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD xqcADHD-`!^ozzy_ozz2321-`!~DinkDink-`!~DinkDonk-`!^mmdfg-`!~PepeLaugh-`!^lutralink-`!~hey that was my car man kinda weird-`!^mestt-`!~!lewd-`!^uhj0-`!~Wokege ITS TIME-`!^schnozebot-`!~⠄⠄⠄⢰⣧⣼⣯⠄⣸⣠⣶⣶⣦⣾⠄⠄⠄⠄⡀⠄⢀⣿⣿⠄⠄⠄⢸⡇⠄⠄ ⠄⠄⠄⣾⣿⠿⠿⠶⠿⢿⣿⣿⣿⣿⣦⣤⣄⢀⡅⢠⣾⣛⡉⠄⠄⠄⠸⢀⣿⠄ ⠄⠄⢀⡋⣡⣴⣶⣶⡀⠄⠄⠙⢿⣿⣿⣿⣿⣿⣴⣿⣿⣿⢃⣤⣄⣀⣥⣿⣿⠄ ⠄⠄⢸⣇⠻⣿⣿⣿⣧⣀⢀⣠⡌⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⠿⠿⠿⣿⣿⣿⠄ ⠄⢀⢸⣿⣷⣤⣤⣤⣬⣙⣛⢿⣿⣿⣿⣿⣿⣿⡿⣿⣿⡍⠄⠄⢀⣤⣄⠉⠋⣰ ⠄⣼⣖⣿⣿⣿⣿⣿⣿⣿⣿⣿⢿⣿⣿⣿⣿⣿⢇⣿⣿⡷⠶⠶⢿⣿⣿⠇⢀⣤ ⠘⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣿⣿⣿⡇⣿⣿⣿⣿⣿⣿⣷⣶⣥⣴⣿⡗ ⢀⠈⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡟⠄ ⢸⣿⣦⣌⣛⣻⣿⣿⣧⠙⠛⠛⡭⠅⠒⠦⠭⣭⡻⣿⣿⣿⣿⣿⣿⣿⣿⡿⠃⠄ ⠘⣿⣿⣿⣿⣿⣿⣿⣿⡆⠄⠄⠄⠄⠄⠄⠄⠄⠹⠈⢋⣽⣿⣿⣿⣿⣵⣾⠃⠄ ⠄⠘⣿⣿⣿⣿⣿⣿⣿⣿⠄⣴⣿⣶⣄⠄⣴⣶⠄⢀⣾⣿⣿⣿⣿⣿⣿⠃⠄⠄ ⠄⠄⠈⠻⣿⣿⣿⣿⣿⣿⡄⢻⣿⣿⣿⠄⣿⣿⡀⣾⣿⣿⣿⣿⣛⠛⠁⠄⠄⠄ ⠄⠄⠄⠄⠈⠛⢿⣿⣿⣿⠁⠞⢿⣿⣿⡄⢿⣿⡇⣸⣿⣿⠿⠛⠁⠄⠄⠄⠄⠄ ⠄⠄⠄⠄⠄⠄⠄⠉⠻⣿⣿⣾⣦⡙⠻⣷⣾⣿⠃⠿⠋⠁⠄⠄⠄⠄⠄⢀⣠⣴ ⣿⣿⣿⣶⣶⣮⣥⣒⠲⢮⣝⡿⣿⣿⡆⣿⡿⠃⠄⠄⠄⠄⠄⠄⠄⣠⣴⣿⣿⣿-`!^krymsonxxxecchi-`!~FORTINAYT OuttaPocket BABACII 💩 IKISBOKISSERISIKIS monkaLaugh BALESTEYSIN FAYVVF 🔔 LUCITIK GIGACHAD HAYBIR IKS IKS IKS IKS ViolinTime GIRAADIFIFDORIDINIII HUH-`!^sean_bah-`!~TrollDespair-`!^h2jojo-`!~DinkDonk-`!^soop_s-`!~RIPBOZO-`!^i_will_juice_xqc-`!~gg-`!^whatdestiny-`!~WutFace WutFace-`!^shricky-`!~unlucky''')

{'chevalier_au_lion_': [0.7446131110191345,
  0.030049746856093407,
  0.07826997339725494,
  0.027682043612003326,
  0.06860858201980591,
  0.05077653378248215,
  1],
 'drov_': [0.7279432415962219,
  0.031951289623975754,
  0.07162153720855713,
  0.03149078041315079,
  0.07684639096260071,
  0.060146763920784,
  1],
 'fixyouract': [0.8806278109550476,
  0.008468607440590858,
  0.03940855339169502,
  0.009509026072919369,
  0.04156610742211342,
  0.02041984722018242,
  1],
 'gokumeister': [0.9168059229850769,
  0.003578816307708621,
  0.03462907299399376,
  0.004803856369107962,
  0.0304335355758667,
  0.009748835116624832,
  1],
 'jasminehollyxo': [0.8806278109550476,
  0.008468607440590858,
  0.03940855339169502,
  0.009509026072919369,
  0.04156610742211342,
  0.02041984722018242,
  1]}

In [ ]:
app = FastAPI()
@app.post('/abcde')
def index(string1):
    print("Yay accepting requests")
    print(string1)
    prediction = get_predictions(string1)  
    print(prediction)
    return {'message':prediction}

server.run_app(app = app)

Public URL: NgrokTunnel: "https://f400-107-167-180-18.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [1168]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Yay accepting requests
-`!^jimmy_thelimmy-`!~!roulette 12-`!^schnozebot-`!~ACTION jimmy_thelimmy lost 12 points in roulette and now has 190 points! xqcSmash-`!~ACTION GameStarioni lost 100 points in roulette and now has 15290 points! xqcSmash-`!~ACTION jcove11 won 3000 points in roulette and now has 20816 points! FeelsGoodMan-`!^gamestarioni-`!~!roulette 100-`!^jcove11-`!~!roulette 3000-`!^mynamesdan-`!~zvch1 zvch2 zvch3-`!^pepolaugh-`!~VeryPog any vod frogs?-`!^arcteryxy-`!~xqKaren-`!^jacobjm_-`!~!roulette all
{'jimmy_thelimmy': [0.8070105314254761, 0.01816815510392189, 0.053306322544813156, 0.02148919738829136, 0.06233138591051102, 0.037694402039051056, 1], 'schnozebot': [0.9042150378227234, 0.003081159433349967, 0.0435403510928154, 0.006464965175837278, 0.035736192017793655, 0.0069622378796339035, 1], 'gamestarioni': [0.7961035370826721, 0.02011748030781746, 0.05837724357843399, 0.023306947201490402, 0.06257454305887222, 0.0395202562212944, 1], 'jcove11': [0.8523862361907959, 

Yay accepting requests
-`!^fossabot-`!~@moldybuns_ xQcOW has been offline for 22 minutes and 26 seconds-`!~@blizzard505 blizzard505 has been following xQcOW for 1 year, 2 months and 16 days-`!^mbay16-`!~full degen cycle-`!^qazwsxedcjimmy-`!~!smp all-`!^blizzard505-`!~!followage-`!^schnozebot-`!~ACTION qazwsxedcjimmy | xqcJuice xqcEZ xqcMood | lost 8 points LuL-`!~ACTION moldybuns_ lost 220 points in roulette and now has 0 points! xqcSmash-`!~ACTION mirrebotti won 2 points in roulette and now has 96 points! FeelsGoodMan-`!~ACTION aimgodayo | xqcWut xqcJuice xqcJuice | won 1399.5 points PogChamp-`!~ACTION cr1m3s666 lost 272 points in roulette and now has 0 points! xqcSmash-`!~ACTION ShadowsWrathXI won 50 points in roulette and now has 1513 points! FeelsGoodMan-`!^moldybuns_-`!~!roulette all-`!~!poof-`!^mirrebotti-`!~!roulette 2-`!^aimgodayo-`!~!slots all-`!^jajajahahdbdbeniw-`!~!slots 10-`!^stankyleg06-`!~I know of someone that made a new acc under a different name and just c

Yay accepting requests
-`!^noivernalchemist-`!~!roulette 1000-`!~!roulette 1000 󠀀-`!^schnozebot-`!~ACTION NoivernAlchemist lost 1000 points in roulette and now has 2442 points! xqcSmash-`!^hannesthelord-`!~xqcN-`!~xqcN 󠀀-`!~xqcN-`!~xqcN 󠀀-`!^0kaychamp_-`!~FeelsBadMan PianoTime-`!^ovoradu-`!~romania-`!^atender-`!~!roulette 5000-`!^fuwwysquiwwei-`!~hannesthelord 󠀀-`!^big_spinich-`!~!farm-`!~!farm 󠀀-`!^viesilver-`!~!roulette 10000
{'noivernalchemist': [0.8356102108955383, 0.014183462597429752, 0.053115375339984894, 0.017464252188801765, 0.05157553404569626, 0.028051190078258514, 1], 'schnozebot': [0.8988450169563293, 0.004419999662786722, 0.04076414555311203, 0.009680720046162605, 0.03600813075900078, 0.010281983762979507, 1], 'hannesthelord': [0.8282102942466736, 0.015605040825903416, 0.07131166756153107, 0.016221091151237488, 0.04459891840815544, 0.024053094908595085, 1], '0kaychamp_': [0.534079909324646, 0.0699593648314476, 0.11480727046728134, 0.07156439125537872, 0.1178290471434593